In [1]:
import openrouteservice
import folium
import pandas as pd
import geopandas as gpd
import pyspark
from pyspark.sql import SparkSession, functions as F
import json
import requests
import regex

In [2]:
rental_df = pd.read_csv("../data/curated/internal_rental.csv")

In [3]:
# Fix the formatting of coordinates
rental_df['coordinates'] = rental_df['coordinates'].str.replace(' ', '').str.replace('[', '').str.replace(']', '')
rental_df['coordinates'] = rental_df['coordinates'].str.split(',')
for i in range(0,len(rental_df['coordinates'])):
    rental_df['coordinates'][i] = [float(x) for x in rental_df['coordinates'][i]]
# Makes all the coordinates scraped into (lattitude ,longitude)--> its actually (longitude, lattitude)
rental_df['coordinates'] = rental_df['coordinates'].apply(lambda x: list(reversed(x)))

/tmp/ipykernel_331/2100596331.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  rental_df['coordinates'] = rental_df['coordinates'].str.replace(' ', '').str.replace('[', '').str.replace(']', '')


In [5]:
rental_df.head()

,name,prop_type,cost_text,coordinates,add_feat,Beds_no,Baths_no,Parking_no,Acreage / Semi-Rural,Apartment / Unit / Flat,...,heating,intercom,internal laundry,no extra features listed,pets allowed,remote garage,secure parking,shed,split system heating,study
0,3309/18 Hoff Boulevard Southbank VIC 3006,Apartment / Unit / Flat,650.0,"[-37.825817, 144.9640414]","['built in wardrobes', 'ducted heating']",2,2,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5605/70 Southbank Boulevard Southbank VIC 3006,Apartment / Unit / Flat,470.0,"[-37.8231931, 144.9639881]","['gas', 'internal laundry', 'intercom', 'floor...",1,1,0,0,1,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,229/22 Kavanagh Street Southbank VIC 3006,Apartment / Unit / Flat,420.0,"[-37.82284060000001, 144.9658569]","['heating', 'balcony deck', 'furnished']",1,1,1,0,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1112/39 Coventry Street Southbank VIC 3006,Apartment / Unit / Flat,550.0,"[-37.8294033, 144.9692407]","['built in wardrobes', 'gas', 'internal laundr...",2,1,1,0,1,...,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
4,1913/39 Coventry Street Southbank VIC 3006,Apartment / Unit / Flat,550.0,"[-37.8294033, 144.9692407]","['air conditioning', 'built in wardrobes', 'ga...",2,1,1,0,1,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Distance and Duration from Melbourne CBD

In [38]:

cbd_dur = []
cbd_dist = []

In [39]:
for i in range(len(rental_df)):
    body = {"coordinates":[rental_df.iloc[i,3],[144.963058,-37.813629]],"instructions":"false","units":"km",'radiuses':[500]}
    headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Content-Type': 'application/json; charset=utf-8'
    }
    call = requests.post('http://localhost:8080/ors/v2/directions/driving-car/json', json=body, headers=headers)
    response_dict = json.loads(call.text)
    # Saves the distance and duration from melbourne cbd if there are routes available, if not, note it down using negative one
    if 'routes' in response_dict.keys():
        cbd_dur.append(response_dict['routes'][0]['summary']['duration']/60)
        cbd_dist.append(response_dict['routes'][0]['summary']['distance']  )
    else:
        cbd_dur.append(-1)
        cbd_dist.append(-1)



In [41]:
rental_df['cbd_dur'] = cbd_dur
rental_df['cbd_dist'] = cbd_dist

In [42]:
rental_df.head()

,name,prop_type,cost_text,coordinates,add_feat,Beds_no,Baths_no,Parking_no,Acreage / Semi-Rural,Apartment / Unit / Flat,...,internal laundry,no extra features listed,pets allowed,remote garage,secure parking,shed,split system heating,study,cbd_dur,cbd_dist
0,3309/18 Hoff Boulevard Southbank VIC 3006,Apartment / Unit / Flat,650.0,"[144.9640414, -37.825817]","['built in wardrobes', 'ducted heating']",2,2,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.615000,2.354
1,5605/70 Southbank Boulevard Southbank VIC 3006,Apartment / Unit / Flat,470.0,"[144.9639881, -37.8231931]","['gas', 'internal laundry', 'intercom', 'floor...",1,1,0,0,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.310000,2.494
2,229/22 Kavanagh Street Southbank VIC 3006,Apartment / Unit / Flat,420.0,"[144.9658569, -37.82284060000001]","['heating', 'balcony deck', 'furnished']",1,1,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.681667,2.328
3,1112/39 Coventry Street Southbank VIC 3006,Apartment / Unit / Flat,550.0,"[144.9692407, -37.8294033]","['built in wardrobes', 'gas', 'internal laundr...",2,1,1,0,1,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,4.933333,2.686
4,1913/39 Coventry Street Southbank VIC 3006,Apartment / Unit / Flat,550.0,"[144.9692407, -37.8294033]","['air conditioning', 'built in wardrobes', 'ga...",2,1,1,0,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.933333,2.686


# Distance and duration from train


In [1]:

from shapely.geometry import Point

In [44]:
sf = gpd.read_file("../data/raw/TRANSPORT/VIC_RAILWAY_STATIONS.shp")

In [45]:
sf['geometry'] = sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
sf.head()

,STATION,STATUS,STOPID_MTR,STID_VLINE,PREMIUM,METRO,VLINE,INTERSTATE,STOP_ZONE,COMMENTS,geometry
0,Dandenong,Active,19888,22250,Yes,Yes,Yes,No,"Reg, 2",Metro Interchange,POINT (145.20964 -37.98992)
1,Darebin,Active,19932,None,No,Yes,No,No,1,None,POINT (145.03863 -37.77480)
2,Darling,Active,19837,None,Yes,Yes,No,No,"1, 2",None,POINT (145.06268 -37.86867)
3,Deer Park,Active,None,19982,No,No,Yes,No,"Reg, 2",None,POINT (144.77122 -37.77750)
4,Dennis,Active,19929,None,No,Yes,No,No,1,None,POINT (145.00871 -37.77918)


In [46]:
#Only keep active stations
active_stations = sf.loc[sf['STATUS'] == 'Active']
active_stations.count()

STATION       314
STATUS        314
STOPID_MTR    220
STID_VLINE    114
PREMIUM       311
METRO         314
VLINE         314
INTERSTATE    314
STOP_ZONE     310
COMMENTS       50
geometry      314
dtype: int64

In [47]:
#find closest coordinate
sf['long'] = sf['geometry'].x
sf['lat'] = sf['geometry'].y

In [49]:
sf

,STATION,STATUS,STOPID_MTR,STID_VLINE,PREMIUM,METRO,VLINE,INTERSTATE,STOP_ZONE,COMMENTS,geometry,long,lat
0,Dandenong,Active,19888,22250,Yes,Yes,Yes,No,"Reg, 2",Metro Interchange,POINT (145.20964 -37.98992),145.209645,-37.989921
1,Darebin,Active,19932,None,No,Yes,No,No,1,None,POINT (145.03863 -37.77480),145.038628,-37.774803
2,Darling,Active,19837,None,Yes,Yes,No,No,"1, 2",None,POINT (145.06268 -37.86867),145.062679,-37.868670
3,Deer Park,Active,None,19982,No,No,Yes,No,"Reg, 2",None,POINT (144.77122 -37.77750),144.771221,-37.777504
4,Dennis,Active,19929,None,No,Yes,No,No,1,None,POINT (145.00871 -37.77918),145.008712,-37.779180
...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,Cranbourne,Active,19886,None,Yes,Yes,No,No,2,Metro Terminus,POINT (145.28001 -38.09916),145.280013,-38.099164
312,Creswick,Active,None,44951,No,No,Yes,No,"Reg, 9",None,POINT (143.88816 -37.42448),143.888160,-37.424484
313,Crib Point,Active,19828,None,No,Yes,No,No,2,None,POINT (145.20398 -38.36593),145.203977,-38.365926
314,Croxton,Active,20015,None,No,Yes,No,No,1,None,POINT (144.99710 -37.76384),144.997105,-37.763841


In [50]:
# Take all coordinates and create a list of it for geojson input:
train_coords = []
for i in range(len(sf)):
    train_coords.append([sf.iloc[i,11],sf.iloc[i,12]])

In [51]:
len_sources = len(train_coords) 

In [52]:
len_sources

316

In [53]:
# These are the headers and inputs used for the 
headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Content-Type': 'application/json; charset=utf-8'
}
base_body = {'locations':train_coords,'metrics':('duration','distance'),'units':'km', 'sources':[len_sources]}


In [57]:
train_prox_2km = []
train_prox_2_min = []
train_prox_closest = []

In [58]:
# Tests whether or not there are train stations within 2 km and 2 minutes of driving time 
for i in range(len(rental_df)):
    base_body['locations'].append(rental_df.iloc[i,3])
    call = requests.post('http://localhost:8080/ors/v2/matrix/driving-car', json=base_body, headers=headers)
    response_dict = json.loads(call.text)
    if 'distances' in response_dict.keys():
        train_prox_2km.append(sum(each_dist <= 2 for each_dist in response_dict['distances'][0]) - 1 )
        train_prox_2_min.append(sum(each_dur <= 120 for each_dur in response_dict['durations'][0]) - 1 )
        train_prox_closest.append(min([dist for dist in response_dict['distances'][0] if dist!=0]))
    else:
        train_prox_2km.append(-1)
        train_prox_2_min.append(-1)
        train_prox_closest.append(-1)
    base_body['locations'].pop()

In [60]:
rental_df['train_prox_2km'] = train_prox_2km
rental_df['train_prox_2_min'] = train_prox_2_min
rental_df['train_prox_closest'] = train_prox_closest


In [61]:
rental_df.head()

,name,prop_type,cost_text,coordinates,add_feat,Beds_no,Baths_no,Parking_no,Acreage / Semi-Rural,Apartment / Unit / Flat,...,remote garage,secure parking,shed,split system heating,study,cbd_dur,cbd_dist,train_prox_2km,train_prox_2_min,train_prox_closest
0,3309/18 Hoff Boulevard Southbank VIC 3006,Apartment / Unit / Flat,650.0,"[144.9640414, -37.825817]","['built in wardrobes', 'ducted heating']",2,2,0,0,1,...,0.0,0.0,0.0,0.0,0.0,5.615000,2.354,2,0,1.42
1,5605/70 Southbank Boulevard Southbank VIC 3006,Apartment / Unit / Flat,470.0,"[144.9639881, -37.8231931]","['gas', 'internal laundry', 'intercom', 'floor...",1,1,0,0,1,...,0.0,0.0,0.0,0.0,0.0,5.310000,2.494,2,0,1.30
2,229/22 Kavanagh Street Southbank VIC 3006,Apartment / Unit / Flat,420.0,"[144.9658569, -37.82284060000001]","['heating', 'balcony deck', 'furnished']",1,1,1,0,1,...,0.0,0.0,0.0,0.0,0.0,4.681667,2.328,1,0,1.13
3,1112/39 Coventry Street Southbank VIC 3006,Apartment / Unit / Flat,550.0,"[144.9692407, -37.8294033]","['built in wardrobes', 'gas', 'internal laundr...",2,1,1,0,1,...,1.0,1.0,0.0,0.0,0.0,4.933333,2.686,1,0,1.49
4,1913/39 Coventry Street Southbank VIC 3006,Apartment / Unit / Flat,550.0,"[144.9692407, -37.8294033]","['air conditioning', 'built in wardrobes', 'ga...",2,1,1,0,1,...,0.0,0.0,0.0,0.0,0.0,4.933333,2.686,1,0,1.49


# Distance And Duration Proximity from  Schools

In [64]:
aus_school = pd.read_csv("../data/raw/australia_school.csv",encoding='cp1252')

In [65]:
vic_school = aus_school[aus_school['Postal_State'] == 'VIC']

In [66]:
vic_school.head(5)

,Education_Sector,Entity_Type,SCHOOL_NO,School_Name,School_Type,School_Status,Address_Line_1,Address_Line_2,Address_Town,Address_State,...,Postal_Address_Line_1,Postal_Address_Line_2,Postal_Town,Postal_State,Postal_Postcode,Full_Phone_No,LGA_ID,LGA_Name,X,Y
0,Government,1,1,Alberton Primary School,Primary,O,21 Thomson Street,NaN,Alberton,VIC,...,21 Thomson Street,NaN,ALBERTON,VIC,3971,03 5183 2412,681,Wellington (S),146.666601,-38.617713
1,Government,1,3,Allansford and District Primary School,Primary,O,Frank Street,NaN,Allansford,VIC,...,Frank Street,NaN,ALLANSFORD,VIC,3277,03 5565 1382,673,Warrnambool (C),142.590393,-38.386281
2,Government,1,4,Avoca Primary School,Primary,O,118 Barnett Street,NaN,Avoca,VIC,...,P O Box 12,NaN,AVOCA,VIC,3467,03 5465 3176,599,Pyrenees (S),143.475649,-37.084502
3,Government,1,8,Avenel Primary School,Primary,O,40 Anderson Street,NaN,Avenel,VIC,...,40 Anderson Street,NaN,AVENEL,VIC,3664,03 5796 2264,643,Strathbogie (S),145.234722,-36.901368
4,Government,1,12,Warrandyte Primary School,Primary,O,5-11 Forbes Street,NaN,Warrandyte,VIC,...,5-11 Forbes Street,NaN,WARRANDYTE,VIC,3113,03 9844 3537,421,Manningham (C),145.213980,-37.742675


In [67]:
# Save the coordinates into a list for inputting
vic_school_coord = []
school_long = vic_school['X'].tolist()
school_lat = vic_school['Y'].tolist()
for i in range(0,len(school_long)):
    vic_school_coord.append([school_long[i],school_lat[i]])

In [69]:
base_body = {'locations':vic_school_coord,'metrics':('duration','distance'),'units':'km', 'sources':[len(vic_school_coord)]}
headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Content-Type': 'application/json; charset=utf-8'
}

In [70]:
school_prox_10km = []
school_prox_2_min = []
school_prox_closest = []

In [71]:
# Tests whether or not there are school within 10 km and 2 minutes of driving time
for i in range(len(rental_df)):
    base_body['locations'].append(rental_df.iloc[i,3])
    call = requests.post('http://localhost:8080/ors/v2/matrix/driving-car', json=base_body, headers=headers)
    response_dict = json.loads(call.text)
    if 'distances' in response_dict.keys():
        school_prox_10km.append(sum(each_dist <= 10 for each_dist in response_dict['distances'][0]) - 1 )
        school_prox_2_min.append(sum(each_dur <= 120 for each_dur in response_dict['durations'][0]) - 1 )
        school_prox_closest.append(min([dist for dist in response_dict['distances'][0] if dist!=0]))
    else:
        school_prox_10km.append(-1)
        school_prox_2_min.append(-1)
        school_prox_closest.append(-1)
    base_body['locations'].pop()

In [77]:
rental_df['school_prox_10km'] = school_prox_10km
rental_df['school_prox_2_min'] = school_prox_2_min
rental_df['school_prox_closest'] = school_prox_closest


In [78]:
rental_df.head()

,name,prop_type,cost_text,coordinates,add_feat,Beds_no,Baths_no,Parking_no,Acreage / Semi-Rural,Apartment / Unit / Flat,...,split system heating,study,cbd_dur,cbd_dist,train_prox_2km,train_prox_2_min,train_prox_closest,school_prox_15km,school_prox_2_min,school_prox_closest
0,3309/18 Hoff Boulevard Southbank VIC 3006,Apartment / Unit / Flat,650.0,"[144.9640414, -37.825817]","['built in wardrobes', 'ducted heating']",2,2,0,0,1,...,0.0,0.0,5.615000,2.354,2,0,1.42,160,0,0.76
1,5605/70 Southbank Boulevard Southbank VIC 3006,Apartment / Unit / Flat,470.0,"[144.9639881, -37.8231931]","['gas', 'internal laundry', 'intercom', 'floor...",1,1,0,0,1,...,0.0,0.0,5.310000,2.494,2,0,1.30,158,0,1.01
2,229/22 Kavanagh Street Southbank VIC 3006,Apartment / Unit / Flat,420.0,"[144.9658569, -37.82284060000001]","['heating', 'balcony deck', 'furnished']",1,1,1,0,1,...,0.0,0.0,4.681667,2.328,1,0,1.13,158,0,0.91
3,1112/39 Coventry Street Southbank VIC 3006,Apartment / Unit / Flat,550.0,"[144.9692407, -37.8294033]","['built in wardrobes', 'gas', 'internal laundr...",2,1,1,0,1,...,0.0,0.0,4.933333,2.686,1,0,1.49,159,0,0.85
4,1913/39 Coventry Street Southbank VIC 3006,Apartment / Unit / Flat,550.0,"[144.9692407, -37.8294033]","['air conditioning', 'built in wardrobes', 'ga...",2,1,1,0,1,...,0.0,0.0,4.933333,2.686,1,0,1.49,159,0,0.85


In [79]:
#rental_df.to_csv("../data/curated/external_rental.csv", index = False)

# Now for Parks

In [80]:
# recreational locations from:https://discover.data.vic.gov.au/dataset/sport-and-recreational-facilities-list, renamed recre_fac
recre_fal = pd.read_excel("../data/raw/recre_fal.xlsx")

In [82]:
len(recre_fal)

9590

In [83]:
recre_fal.columns

Index(['Facility ID', 'LGA Name', 'Facility Name', 'Facility_AutoNumber',
       'Street #', 'Street Name', 'Street Type', 'Suburb/Town', 'Pcode',
       'Melway Ref', 'VicRoads Ref', 'Latitude', 'Longitude',
       'Facility Ownership', 'Facility Purpose', 'Facility Category',
       'CFA Safer Place?', 'Access_', 'FaciltySportPlayedID', 'Facility ID.1',
       'Sports Played', 'Number of Field/Courts', 'Field/Surface Type',
       'Age of Facility', 'Condition of Facility', 'Facility Upgrade Age',
       'Changerooms', 'Facility Features',
       'Spectator numbers for seating/shelter', 'FullAddress',
       'MelwaysVicRoadsRef'],
      dtype='object')

In [84]:
keep = ['Suburb/Town', 'Pcode','Latitude', 'Longitude']
recre_fal_df = pd.DataFrame(recre_fal,columns= keep)

In [85]:
recre_fal_df = recre_fal_df.dropna()

In [86]:
len(recre_fal_df)

8839

In [87]:
# Removing datapoints which had incorrect lattitude coordinate
recre_fal_df = recre_fal_df.drop(recre_fal_df[recre_fal_df['Latitude'] < -40].index)

In [88]:
# Store the coordinates into list
recre_coord = []
recre_long = recre_fal_df['Longitude'].tolist()
recre_lat = recre_fal_df['Latitude'].tolist()
for i in range(0,len(recre_long)):
    recre_coord.append([recre_long[i],recre_lat[i]])

In [90]:
recre_prox_15km = []
recre_prox_2_min = []
recre_prox_closest = []

In [91]:
len(recre_coord)

8837

In [92]:
base_body = {'locations':recre_coord,'metrics':('duration','distance'),'units':'km', 'sources':[len(recre_coord)]}
headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Content-Type': 'application/json; charset=utf-8'
}

In [93]:
# Tests whether or not there are recreational facilities within 15 km and 2 minutes of driving time, as well as closest recreational facility
for i in range(len(rental_df)):
    base_body['locations'].append(rental_df.iloc[i,3])
    call = requests.post('http://localhost:8080/ors/v2/matrix/driving-car', json=base_body, headers=headers)
    response_dict = json.loads(call.text)
    if 'distances' in response_dict.keys():
        recre_prox_15km.append(sum(each_dist <= 15 for each_dist in response_dict['distances'][0]) - 1 )
        recre_prox_2_min.append(sum(each_dur <= 120 for each_dur in response_dict['durations'][0]) - 1 )
        recre_prox_closest.append(min([dist for dist in response_dict['distances'][0] if dist!=0]))
    else:
        recre_prox_15km.append(-1)
        recre_prox_2_min.append(-1)
        recre_prox_closest.append(-1)
    base_body['locations'].pop()

In [95]:
rental_df['recre_prox_15km'] = recre_prox_15km
rental_df['recre_prox_2_min'] = recre_prox_2_min
rental_df['recre_prox_closest'] = recre_prox_closest

In [100]:
rental_df

,name,prop_type,cost_text,coordinates,add_feat,Beds_no,Baths_no,Parking_no,Acreage / Semi-Rural,Apartment / Unit / Flat,...,cbd_dist,train_prox_2km,train_prox_2_min,train_prox_closest,school_prox_15km,school_prox_2_min,school_prox_closest,recre_prox_15km,recre_prox_2_min,recre_prox_closest
0,3309/18 Hoff Boulevard Southbank VIC 3006,Apartment / Unit / Flat,650.0,"[144.9640414, -37.825817]","['built in wardrobes', 'ducted heating']",2,2,0,0,1,...,2.354,2,0,1.42,160,0,0.76,1010,0,1.25
1,5605/70 Southbank Boulevard Southbank VIC 3006,Apartment / Unit / Flat,470.0,"[144.9639881, -37.8231931]","['gas', 'internal laundry', 'intercom', 'floor...",1,1,0,0,1,...,2.494,2,0,1.30,158,0,1.01,1009,0,1.27
2,229/22 Kavanagh Street Southbank VIC 3006,Apartment / Unit / Flat,420.0,"[144.9658569, -37.82284060000001]","['heating', 'balcony deck', 'furnished']",1,1,1,0,1,...,2.328,1,0,1.13,158,0,0.91,1023,0,1.37
3,1112/39 Coventry Street Southbank VIC 3006,Apartment / Unit / Flat,550.0,"[144.9692407, -37.8294033]","['built in wardrobes', 'gas', 'internal laundr...",2,1,1,0,1,...,2.686,1,0,1.49,159,0,0.85,1000,0,2.00
4,1913/39 Coventry Street Southbank VIC 3006,Apartment / Unit / Flat,550.0,"[144.9692407, -37.8294033]","['air conditioning', 'built in wardrobes', 'ga...",2,1,1,0,1,...,2.686,1,0,1.49,159,0,0.85,1000,0,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13033,20 Tarooh Street Cape Paterson VIC 3995,House,500.0,"[145.6181208, -38.67034659999999]","['secure parking', 'bath', 'dishwasher', 'shed']",4,2,0,0,0,...,-1.000,0,0,82.53,3,0,7.68,38,3,0.54
13034,1/5 Sir Henry Bolte Court Wonthaggi VIC 3995,House,380.0,"[145.5891883, -38.6001766]","['gas', 'secure parking']",2,1,1,0,0,...,-1.000,0,0,74.18,6,0,1.18,41,0,1.13
13035,3/37 Dunn Street Wonthaggi VIC 3995,Apartment / Unit / Flat,330.0,"[145.5892931, -38.6137663]","['built in wardrobes', 'internal laundry', 'he...",2,1,1,0,1,...,-1.000,0,0,75.21,6,1,0.81,41,2,0.45
13036,59 Billson Street Wonthaggi VIC 3995,House,480.0,"[145.5970984, -38.6151218]","['internal laundry', 'secure parking', 'heating']",3,2,0,0,0,...,-1.000,0,0,75.49,6,1,0.65,40,2,1.20


# Shopping Centre


In [10]:
# Shopping centres: Collected from querying shops/food, convience shops and general shops in qgis
shopping_sf = gpd.read_file("../data/raw/shopping_data/general_shopping.shp")

In [11]:
shopping_sf.head()

,full_id,osm_id,osm_type,shop,not_brand_,highway,start_date,image,opening_ho,diet_koshe,...,opening__3,is_in,wheelchair,postal_cod,brand_wiki,brand_wi_1,brand,branch,name,geometry
0,n32193447,32193447,node,supermarket,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,IGA,POINT (144.73386 -37.27736)
1,n48670894,48670894,node,supermarket,None,None,None,None,None,None,...,None,None,yes,3152,en:Ritchies Stores,Q7336696,Ritchies,Supa IGA,Ritchies,POINT (145.23306 -37.84776)
2,n218028470,218028470,node,supermarket,None,None,None,None,None,None,...,07:00-24:00,"East Doncaster,Victoria,Australia",None,3109,en:Coles Supermarkets,Q1108172,Coles,None,Coles,POINT (145.17184 -37.79143)
3,n222420863,222420863,node,supermarket,None,None,None,None,None,None,...,24/7,None,no,None,en:FoodWorks,Q5465579,Foodworks,Carlton North,Foodworks Carlton North,POINT (144.97609 -37.78899)
4,n243975028,243975028,node,convenience,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,POINT (143.77512 -38.01930)


In [12]:
# Fix formatting
shopping_sf['geometry'] = shopping_sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
shopping_sf['long'] = shopping_sf['geometry'].x
shopping_sf['lat'] = shopping_sf['geometry'].y
shopping_sf.head()

,full_id,osm_id,osm_type,shop,not_brand_,highway,start_date,image,opening_ho,diet_koshe,...,wheelchair,postal_cod,brand_wiki,brand_wi_1,brand,branch,name,geometry,long,lat
0,n32193447,32193447,node,supermarket,None,None,None,None,None,None,...,None,None,None,None,None,None,IGA,POINT (144.73386 -37.27736),144.733861,-37.277362
1,n48670894,48670894,node,supermarket,None,None,None,None,None,None,...,yes,3152,en:Ritchies Stores,Q7336696,Ritchies,Supa IGA,Ritchies,POINT (145.23306 -37.84776),145.233059,-37.847757
2,n218028470,218028470,node,supermarket,None,None,None,None,None,None,...,None,3109,en:Coles Supermarkets,Q1108172,Coles,None,Coles,POINT (145.17184 -37.79143),145.171842,-37.791434
3,n222420863,222420863,node,supermarket,None,None,None,None,None,None,...,no,None,en:FoodWorks,Q5465579,Foodworks,Carlton North,Foodworks Carlton North,POINT (144.97609 -37.78899),144.976090,-37.788987
4,n243975028,243975028,node,convenience,None,None,None,None,None,None,...,None,None,None,None,None,None,None,POINT (143.77512 -38.01930),143.775117,-38.019295


In [13]:
shopping_sf.head()

,full_id,osm_id,osm_type,shop,not_brand_,highway,start_date,image,opening_ho,diet_koshe,...,wheelchair,postal_cod,brand_wiki,brand_wi_1,brand,branch,name,geometry,long,lat
0,n32193447,32193447,node,supermarket,None,None,None,None,None,None,...,None,None,None,None,None,None,IGA,POINT (144.73386 -37.27736),144.733861,-37.277362
1,n48670894,48670894,node,supermarket,None,None,None,None,None,None,...,yes,3152,en:Ritchies Stores,Q7336696,Ritchies,Supa IGA,Ritchies,POINT (145.23306 -37.84776),145.233059,-37.847757
2,n218028470,218028470,node,supermarket,None,None,None,None,None,None,...,None,3109,en:Coles Supermarkets,Q1108172,Coles,None,Coles,POINT (145.17184 -37.79143),145.171842,-37.791434
3,n222420863,222420863,node,supermarket,None,None,None,None,None,None,...,no,None,en:FoodWorks,Q5465579,Foodworks,Carlton North,Foodworks Carlton North,POINT (144.97609 -37.78899),144.976090,-37.788987
4,n243975028,243975028,node,convenience,None,None,None,None,None,None,...,None,None,None,None,None,None,None,POINT (143.77512 -38.01930),143.775117,-38.019295


In [14]:
# Take all coordinates and create a list of it for geojson input:
shop_coords = []
for i in range(len(shopping_sf)):
    shop_coords.append([shopping_sf.iloc[i,105],shopping_sf.iloc[i,106]])

In [15]:
headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Content-Type': 'application/json; charset=utf-8'
}
base_body = {'locations':shop_coords,'metrics':('duration','distance'),'units':'km', 'sources':[len(shop_coords)]}

In [16]:
shopping_prox_5km = []
shopping_prox_2_min = []
shopping_prox_closest = []

In [18]:
# Tests whether or not there are shopping centres within 5 km and 2 minutes of driving time 
for i in range(len(rental_df)):
    base_body['locations'].append(rental_df.iloc[i,3])
    call = requests.post('http://localhost:8080/ors/v2/matrix/driving-car', json=base_body, headers=headers)
    response_dict = json.loads(call.text)
    if 'distances' in response_dict.keys():
        shopping_prox_5km.append(sum(each_dist <= 5 for each_dist in response_dict['distances'][0]) - 1 )
        shopping_prox_2_min.append(sum(each_dur <= 120 for each_dur in response_dict['durations'][0]) - 1 )
        shopping_prox_closest.append(min([dist for dist in response_dict['distances'][0] if dist!=0]))
    else:
        shopping_prox_5km.append(-1)
        shopping_prox_2_min.append(-1)
        shopping_prox_closest.append(-1)
    base_body['locations'].pop()

In [20]:
shopping_prox_5km_df = pd.DataFrame(shopping_prox_5km, columns = ['shopping_prox_5km'])
shopping_prox_2_min_df = pd.DataFrame(shopping_prox_2_min, columns = ['shopping_prox_2_min'])
shopping_prox_closest_df = pd.DataFrame(shopping_prox_closest, columns = ['shopping_prox_closest'])

# Proximity to Hospitals

In [22]:
# Shopping centres: Collected from querying hospitals in qgis
hosp_sf = gpd.read_file("../data/raw/hospital_data/hospital.shp")

In [31]:
hosp_sf.head()

,full_id,osm_id,osm_type,amenity,healthcare,phone_AU,wheelchair,url,insurance_,health_ame,...,addr_stree,addr_state,addr_postc,addr_hou_1,website,name,emergency,geometry,long,lat
0,n27288796,27288796,node,hospital,None,None,None,None,None,None,...,None,None,None,None,https://www.waverleyprivate.com.au/,Waverley Private Hospital,no,POINT (145.14557 -37.88438),145.145572,-37.884379
1,n206200909,206200909,node,hospital,None,None,None,None,None,None,...,None,None,None,None,None,Manangatang Hospital,None,POINT (142.88083 -35.05157),142.880825,-35.051571
2,n262777337,262777337,node,hospital,hospital,None,None,None,None,None,...,Burwood Highway,VIC,3151,321-345,https://www.easternhealth.org.au/locations/pet...,Peter James Centre,no,POINT (145.16495 -37.85261),145.164949,-37.852615
3,n429184771,429184771,node,hospital,None,None,None,None,None,None,...,None,None,None,None,None,Jessie McPherson Private Hospital,None,POINT (145.12248 -37.92027),145.122482,-37.920267
4,n489359423,489359423,node,hospital,hospital,None,None,None,None,None,...,Brettoneaux Street,None,3717,1,www.seymourhealth.org.au,Seymour Hospital,None,POINT (145.13820 -37.01852),145.138196,-37.018515
5,n597972749,597972749,node,hospital,None,None,None,None,None,None,...,None,None,None,None,None,Emergency,None,POINT (145.21272 -37.93804),145.212720,-37.938036
6,n600761782,600761782,node,hospital,hospital,None,None,None,None,None,...,Cobden Street,None,3741,36-42,None,Bright Hospital,yes,POINT (146.96597 -36.73296),146.965973,-36.732956
7,n687479687,687479687,node,hospital,None,None,None,None,None,None,...,None,None,None,None,None,Timboon District Healthcare Service,None,POINT (142.97650 -38.48607),142.976497,-38.486075
8,n701715127,701715127,node,hospital,None,None,None,None,None,None,...,None,None,None,None,None,Bannockburn Aged Care,None,POINT (144.16531 -38.03748),144.165309,-38.037477
9,n704426805,704426805,node,hospital,None,None,None,None,None,None,...,None,None,None,None,None,Terang & Mortlake Health Service,None,POINT (142.90206 -38.23939),142.902058,-38.239387


In [24]:
# Fix formatting
hosp_sf['geometry'] = hosp_sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
hosp_sf['long'] = hosp_sf['geometry'].x
hosp_sf['lat'] = hosp_sf['geometry'].y
hosp_sf.head()

,full_id,osm_id,osm_type,amenity,healthcare,phone_AU,wheelchair,url,insurance_,health_ame,...,addr_stree,addr_state,addr_postc,addr_hou_1,website,name,emergency,geometry,long,lat
0,n27288796,27288796,node,hospital,None,None,None,None,None,None,...,None,None,None,None,https://www.waverleyprivate.com.au/,Waverley Private Hospital,no,POINT (145.14557 -37.88438),145.145572,-37.884379
1,n206200909,206200909,node,hospital,None,None,None,None,None,None,...,None,None,None,None,None,Manangatang Hospital,None,POINT (142.88083 -35.05157),142.880825,-35.051571
2,n262777337,262777337,node,hospital,hospital,None,None,None,None,None,...,Burwood Highway,VIC,3151,321-345,https://www.easternhealth.org.au/locations/pet...,Peter James Centre,no,POINT (145.16495 -37.85261),145.164949,-37.852615
3,n429184771,429184771,node,hospital,None,None,None,None,None,None,...,None,None,None,None,None,Jessie McPherson Private Hospital,None,POINT (145.12248 -37.92027),145.122482,-37.920267
4,n489359423,489359423,node,hospital,hospital,None,None,None,None,None,...,Brettoneaux Street,None,3717,1,www.seymourhealth.org.au,Seymour Hospital,None,POINT (145.13820 -37.01852),145.138196,-37.018515


In [29]:
# Take all coordinates and create a list of it for geojson input:
hosp_coords = []
for i in range(len(hosp_sf)):
    hosp_coords.append([hosp_sf.iloc[i,38],hosp_sf.iloc[i,39]])

In [33]:
headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Content-Type': 'application/json; charset=utf-8'
}
base_body = {'locations':hosp_coords,'metrics':('duration','distance'),'units':'km', 'sources':[len(hosp_coords)]}

In [34]:
hosp_prox_5km = []
hosp_closest = []

In [35]:
# Tests whether or not there are hospitals within 5km of driving and how far the closest one is
for i in range(len(rental_df)):
    base_body['locations'].append(rental_df.iloc[i,3])
    call = requests.post('http://localhost:8080/ors/v2/matrix/driving-car', json=base_body, headers=headers)
    response_dict = json.loads(call.text)
    if 'distances' in response_dict.keys():
        hosp_prox_5km.append(sum(each_dist <= 5 for each_dist in response_dict['distances'][0]) - 1 )
        hosp_closest.append(min([dist for dist in response_dict['distances'][0] if dist!=0]))
    else:
        hosp_prox_5km.append(-1)
        hosp_closest.append(-1)
    base_body['locations'].pop()

In [40]:
rental_df['hosp_prox_5km'] = hosp_prox_5km
rental_df['hosp_closest'] = hosp_closest


In [41]:
rental_df.head()

,name,prop_type,cost_text,coordinates,add_feat,Beds_no,Baths_no,Parking_no,Acreage / Semi-Rural,Apartment / Unit / Flat,...,remote garage,secure parking,shed,split system heating,study,shopping_prox_5km,shopping_prox_2_min,shopping_prox_closest,hosp_prox_5km,hosp_closest
0,3309/18 Hoff Boulevard Southbank VIC 3006,Apartment / Unit / Flat,650.0,"[144.9640414, -37.825817]","['built in wardrobes', 'ducted heating']",2,2,0,0,1,...,0.0,0.0,0.0,0.0,0.0,197,8,0.02,5,3.07
1,5605/70 Southbank Boulevard Southbank VIC 3006,Apartment / Unit / Flat,470.0,"[144.9639881, -37.8231931]","['gas', 'internal laundry', 'intercom', 'floor...",1,1,0,0,1,...,0.0,0.0,0.0,0.0,0.0,197,8,0.02,5,2.95
2,229/22 Kavanagh Street Southbank VIC 3006,Apartment / Unit / Flat,420.0,"[144.9658569, -37.82284060000001]","['heating', 'balcony deck', 'furnished']",1,1,1,0,1,...,0.0,0.0,0.0,0.0,0.0,197,8,0.02,5,2.78
3,1112/39 Coventry Street Southbank VIC 3006,Apartment / Unit / Flat,550.0,"[144.9692407, -37.8294033]","['built in wardrobes', 'gas', 'internal laundr...",2,1,1,0,1,...,1.0,1.0,0.0,0.0,0.0,197,7,0.02,5,3.14
4,1913/39 Coventry Street Southbank VIC 3006,Apartment / Unit / Flat,550.0,"[144.9692407, -37.8294033]","['air conditioning', 'built in wardrobes', 'ga...",2,1,1,0,1,...,0.0,0.0,0.0,0.0,0.0,197,7,0.02,5,3.14


In [43]:
rental_df.to_csv("../data/curated/proximity_calc_final.csv", index = False)